In [1]:
import csv
import ast
import numpy as np
import time
import random
import math

start = time.time()

# --- Load BB (Billboard data) ---
BB = []
with open("BB_LA.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        BB.append({'B_id': int(row['B_index'])})

# --- Load ub (User influence data) ---
ub = []
with open("new_ub.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        row['Influenced Billboards'] = ast.literal_eval(row['Influenced Billboards'])
        ub.append(row)

# --- Load Dem (Demand values) ---
Dem = []
with open("Advertiser_LA1.csv", 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        Dem.append(float(row['Demand']))

# BillboardSlot class
class BillboardSlot:
    def __init__(self, idx):
        self.idx = idx

    def __repr__(self):
        return f"Slot({self.idx})"

# Convert BB to billboard slots
BS = [BillboardSlot(item['B_id']) for item in BB]
random_integers = np.random.randint(1, 100, size=len(BS))
x = random_integers / 100.0

In [2]:
# Influence function
def influence(S):
    influence_score = 0.0
    slot_ids = {s.idx if isinstance(s, BillboardSlot) else s for s in S}
    for row in ub:
        influenced_billboards = row.get('Influenced Billboards', {})
        intersection_result = {int(key): value for key, value in influenced_billboards.items() if int(key) in slot_ids}
        product = 1.0
        for value in intersection_result.values():
            product *= (1 - value)
        influence_score += (1 - product)
    return influence_score

In [3]:
# Multilinear extension
def multilinear_extension(y):
    n = len(y)
    F_x = 0.0
    # print("\n--------------- Generating 5 Random Subsets from BS and Calculating Terms ------------")
    no_of_subsets = 5
    sampled_subsets = []
    for _ in range(no_of_subsets):
        subset_size = random.randint(1, len(BS))
        subset = tuple(random.sample(BS, subset_size))
        # print("subsets",subset)
        sampled_subsets.append(subset)
    # print("Sampled_subsets",sampled_subsets)
    for S_values in sampled_subsets:
        f_S = influence(S_values)
        # print("Influence:",f_S)
        indices_S = [i for i in range(n) if BS[i] in S_values]
        # print("indices of S",indices_S)
        indices_not_S = [i for i in range(n) if BS[i] not in S_values]
        # print("indices not in S",indices_not_S)
        prod_S = np.prod([x[i] for i in indices_S]) if indices_S else 1.0
        prod_not_S = np.prod([1 - x[i] for i in indices_not_S]) if indices_not_S else 1.0
        if prod_not_S == 0.0:
            prod_not_S = 0.001
        # print("prod_S",prod_S)
        # print("prod_not_S",prod_not_S)
        term = f_S * prod_S * prod_not_S
        F_x += term

    return F_x

In [4]:
# One-hot
def one_val(index, size):
    vec = np.zeros(size)
    vec[index] = 1
    return vec

# Marginal gain
def compute_marginal_gain(e, y):
    y_or_1e = np.maximum(y, one_val(e, len(y)))
    a = multilinear_extension(y_or_1e)
    # print("value of a = ", a)
    b = multilinear_extension(y)
    # print("value of b = ", b)
    return a - b


In [5]:
# Solve LP
def solve_polymatroid_lp(w, n):
    I_t = np.zeros(n)
    sorted_indices = np.argsort(w)[::-1]
    for i in sorted_indices[:n // 2]:
        I_t[i] = 1.0
    return I_t

# Continuous Greedy
def continuous_greedy(BS):
    T = 1
    n = len(BS)
    # delta = T/math.ceil(n**5*T)
    delta = 0.4
    y = np.zeros(n)
    t = 0.0
    while t < T:
        print(f"\n=== Time step t = {t} ===")
        w = [compute_marginal_gain(e, y) for e in range(n)]
        I_t = solve_polymatroid_lp(w, n)
        for e in range(n):
            y[e] += delta * I_t[e] * (1 - y[e])
        t += delta
        # print('t:',t)
    return y


In [6]:
#Calcualtion for value of k in STOCHASTIC-GREEDY
def value_of_k(BS, Dem):
    n = len(BS)
    # print("length of BS",n)
    max_demand = max(Dem)
    # print("Maximum demand:", max_demand)

    bs_i = set()
    available_slots = {BS[i].idx for i in range(n)}

    while influence(bs_i) < max_demand:
        max_gain = -float('inf')
        u_star = None
        for u in available_slots - bs_i:
            gain = influence(bs_i | {u}) - influence(bs_i)
            if gain > max_gain:
                max_gain = gain
                u_star = u
        if u_star is None:
            break
        bs_i.add(u_star)
    # print("influence of bs_i",influence(bs_i))

    return len(bs_i)

In [7]:
#Bi-Criteria_Approximation
def bi_criteria_approximation(BS, r, epsilon):
    h = len(Dem)
    n = len(BS)
    k = value_of_k(BS,Dem)
    t = int((n / k) * math.log(1 / epsilon))
    # print("value of k",k)
    Z = continuous_greedy(BS)
    # print("value of Z",Z)
    alpha = 1 - epsilon
    l = math.ceil(math.log(1 / alpha) * r)
    BS = [slot.idx for slot in BS]
    # Define an empty set S for storing the samples
    S = set()
    for _ in range(l):
        s = set()
        for i in range(n):
            if random.random() < Z[i]:
                s.add(BS[i])
        S.update(s)
    # print("value of S",S)
    # print("no of elements in S",len(S))
    # print("influence of S",influence(S))
    # Check constraints for each advertiser j
    for j in range(h):
        demand_j = Dem[j]
        I_j_S = influence(S)
        required_influence = (1 - (1 / math.e) - 2 * epsilon) * demand_j

        if I_j_S < required_influence:
            M_j = set()

            while influence(M_j) < (1 - (1 / math.e)) * demand_j:
                # print("influence of M_j",influence(M_j))
                # Sample a subset R of size t from BS
                sampled_R = random.sample(BS, min(t, len(BS)))
                # print(f"Sampled R size: {len(sampled_R)}")
                # print(f"Sampled R : {sampled_R}")

                # Find u that maximizes marginal gain
                max_gain = -float('inf')
                u_star = None
                for u in sampled_R:
                    # print('u:',u)
                    a = influence(M_j)
                    M_j.add(u)
                    b = influence(M_j)
                    gain = b-a
                    M_j.remove(u)
                    # print('Gain:',gain)
                    # print('Maxgain:',max_gain)
                    if gain > max_gain:
                        max_gain = gain
                        u_star = u
                if u_star is None:
                    break 
                M_j.add(u_star)
                # print('MJ',M_j)
                for element in M_j:
                    if element in BS:
                        BS.remove(element)
                # print('length of BS',len(BS))
            # print("Influence of M_j finally",influence(M_j))
            # print("Demand ",demand_j)
            # print(f"Final M_j size for advertiser {j}: {len(M_j)}")
            S.update(M_j)

    return S


In [8]:
# Run
r = len(BS)
epsilon = 0.1
result = bi_criteria_approximation(BS, r, epsilon)
# print("Selected billboard slots (by index):", result)
print("Influence : ", influence(result))
print("no of result slots:",len(result))

elapsed_time = time.time() - start
hours, rem = divmod(elapsed_time, 3600)
minutes, seconds = divmod(rem, 60)

print(f"Execution time: {int(hours)}h {int(minutes)}m {seconds:.2f}")
print(f"Execution time: {time.time() - start:.2f} seconds")



=== Time step t = 0.0 ===

=== Time step t = 0.4 ===

=== Time step t = 0.8 ===
Influence :  1770.1610688442333
no of result slots: 4125
Execution time: 3h 28m 29.39
Execution time: 12509.39 seconds


In [9]:
h = len(Dem)
count = 0
for j in range(h):
    if influence(result) > Dem[j]:
        count = count+1
print("no of products whose demands are got satisfied :",count )   

no of products whose demands are got satisfied : 100
